In [181]:
import pandas as pd
import jsonpickle
import tweepy
import json
import unidecode
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

In [182]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 999)

In [183]:
tweets_data_path_si = 'data/tweets_si.json'

tweets_file_si = open(tweets_data_path_si, "r",encoding="utf8")
tweets_data_si = json.load(tweets_file_si)

In [184]:
tweets_data_si[0]

{'contributors': None,
 'truncated': False,
 'text': '#AbortoLegal https://t.co/uKIc4txUI9',
 'is_quote_status': True,
 'in_reply_to_status_id': None,
 'id': 986209816115892225,
 'favorite_count': 0,
 'entities': {'symbols': [],
  'user_mentions': [],
  'hashtags': [{'indices': [0, 12], 'text': 'AbortoLegal'}],
  'urls': [{'url': 'https://t.co/uKIc4txUI9',
    'indices': [13, 36],
    'expanded_url': 'https://twitter.com/infobae/status/986031488301924352',
    'display_url': 'twitter.com/infobae/status…'}]},
 'quoted_status_id': 986031488301924352,
 'retweeted': False,
 'coordinates': None,
 'quoted_status': {'contributors': None,
  'truncated': False,
  'text': 'El día que René Favaloro se pronunció a favor de la legalización del aborto https://t.co/gjZE8i0UVE https://t.co/QV6SXP1zat',
  'is_quote_status': False,
  'in_reply_to_status_id': None,
  'id': 986031488301924352,
  'favorite_count': 530,
  'entities': {'symbols': [],
   'user_mentions': [],
   'hashtags': [],
   'urls': [{'u

In [185]:
def recortar_tweet(t):
    #COLUMNAS = ['created_at', 'favorite_count', 'retweet_count', 'text', 'source']
    COLUMNAS = [ 'text']
    tweet_recortado = {columna: t[columna] for columna in COLUMNAS}
    #tweet_recortado['user_name'] = t['user']['name']
    #tweet_recortado['user_location'] = t['user']['location']
    #tweet_recortado['url'] = t['user']['location']
    return tweet_recortado

In [186]:
recortados = [recortar_tweet(t) for t in tweets_data_si]
tweets_si = pd.DataFrame(recortados)

In [187]:
tweets_si

,text
0,#AbortoLegal https://t.co/uKIc4txUI9
1,RT @estipozzo: Veo los listados de expositores de mañana por #AbortoLegal. Se viene repitiendo lo siguiente: a favor suelen ser mayoría muj…
2,RT @SofiBotto: HOY! Nuestra coordinadora nacional @raquelvivanco expone en la Cámara de Diputadxs a favor de la ley de #AbortoLegal. \nA la…
3,RT @SofiBotto: HOY! Nuestra coordinadora nacional @raquelvivanco expone en la Cámara de Diputadxs a favor de la ley de #AbortoLegal. \nA la…
4,RT @raquelvivanco: Mañana me toca. #AbortoLegal #AhoraEsCuando 💪🏽💚 https://t.co/TackkRzVb4
5,"RT @Red__Accion: #CABA #HOY Tuitazo y actividades en el Congreso por el Aborto Legal, Seguro y Gratuito https://t.co/RnF00ZuDuJ \n#AbortoLeg…"
6,"RT @ingridurrut: #buenlunes #ATR x el #AbortoLegal Basta de juzgarnos, basta de perseguirnos hoy más que nunca redoblemos la presencia en l…"
7,"RT @FranjaMendoza: El Rector @ingdanielpizzi\nadhirió a la declaración emitida por el CIN a favor del #Abortolegal, seguro y gratuito.\n\nhttp…"
8,"#Argentina #AbortoLegal ""Mi hija no podrá debatir"" https://t.co/Gy67Pspbbh Por @gargantapodero @Red__Accion https://t.co/salDiP4BRs"
9,RT @TatianaGoransky: Madrid. Hoy ¡Mucha ilu! Presentación doble + amigos + firma a la distancia por #AbortoLegal en Argentina. Gran día. ht…


In [188]:
tweets_si['posicion'] = 'si'

In [189]:
tweets_si

,text,posicion
0,#AbortoLegal https://t.co/uKIc4txUI9,si
1,RT @estipozzo: Veo los listados de expositores de mañana por #AbortoLegal. Se viene repitiendo lo siguiente: a favor suelen ser mayoría muj…,si
2,RT @SofiBotto: HOY! Nuestra coordinadora nacional @raquelvivanco expone en la Cámara de Diputadxs a favor de la ley de #AbortoLegal. \nA la…,si
3,RT @SofiBotto: HOY! Nuestra coordinadora nacional @raquelvivanco expone en la Cámara de Diputadxs a favor de la ley de #AbortoLegal. \nA la…,si
4,RT @raquelvivanco: Mañana me toca. #AbortoLegal #AhoraEsCuando 💪🏽💚 https://t.co/TackkRzVb4,si
5,"RT @Red__Accion: #CABA #HOY Tuitazo y actividades en el Congreso por el Aborto Legal, Seguro y Gratuito https://t.co/RnF00ZuDuJ \n#AbortoLeg…",si
6,"RT @ingridurrut: #buenlunes #ATR x el #AbortoLegal Basta de juzgarnos, basta de perseguirnos hoy más que nunca redoblemos la presencia en l…",si
7,"RT @FranjaMendoza: El Rector @ingdanielpizzi\nadhirió a la declaración emitida por el CIN a favor del #Abortolegal, seguro y gratuito.\n\nhttp…",si
8,"#Argentina #AbortoLegal ""Mi hija no podrá debatir"" https://t.co/Gy67Pspbbh Por @gargantapodero @Red__Accion https://t.co/salDiP4BRs",si
9,RT @TatianaGoransky: Madrid. Hoy ¡Mucha ilu! Presentación doble + amigos + firma a la distancia por #AbortoLegal en Argentina. Gran día. ht…,si


In [190]:
tweets_data_path_no = 'data/tweets_no.json'

tweets_file_no = open(tweets_data_path_no, "r",encoding="utf8")
tweets_data_no = json.load(tweets_file_no)

In [191]:
recortados = [recortar_tweet(t) for t in tweets_data_no]
tweets_no = pd.DataFrame(recortados)

In [192]:
tweets_no['posicion'] = 'no'

In [193]:
frames =[tweets_si,tweets_no]

In [194]:
df = pd.concat(frames)

In [195]:
df.columns

Index(['text', 'posicion'], dtype='object')

In [196]:
df.to_csv('tweetsaborto.csv', index=False, encoding='utf8')

# Analisis de los datos

In [197]:
df_tweets = pd.read_csv('tweetsaborto.csv', encoding='utf8')

In [198]:
df_tweets = df_tweets.astype(str)

In [199]:
df_tweets.dtypes

text        object
posicion    object
dtype: object

In [201]:
stopwords = pd.read_csv('stopwords.csv', header= None)

In [202]:
df_tweets.describe()

,text,posicion
count,4313,4313
unique,1610,3
top,RT @nbg__: Nicolás Massot sobre el #AbortoLegal en LN. https://t.co/ItV5gNBEjF,si
freq,105,2462


In [203]:
df_tweets = df_tweets[pd.notnull(df_tweets['text'])]

In [204]:
stopwords_list = stopwords[0].values
stopwords_list = list(stopwords_list)

In [205]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [206]:
def text_stemmer(text):
    if type(text) is str:
        texto = text.lower()
        unaccented_string = unidecode.unidecode(texto)
        stripped = re.sub('[^a-z0-9]+', ' ', unaccented_string).lstrip()
        stemmed = " ".join([stemmer.stem(x) for x in stripped.split(' ')])
        return stemmed

In [207]:
import unidecode
import re
df_tweets['text2'] = df_tweets["text"].apply(text_stemmer)
df_tweets = df_tweets[pd.notnull(df_tweets['text2'])]

In [208]:
df_tweets.describe()

,text,posicion,text2
count,4313,4313,4313
unique,1610,3,1610
top,RT @nbg__: Nicolás Massot sobre el #AbortoLegal en LN. https://t.co/ItV5gNBEjF,si,rt nbg nicol massot sobr el abortolegal en ln https t co itv5gnbejf
freq,105,2462,105


In [209]:
df_tweets.groupby('posicion')['text'].count()

posicion
nan    4   
no     1847
si     2462
Name: text, dtype: int64

In [210]:
df_tweets.dropna( how='any') 

,text,posicion,text2
0,#AbortoLegal https://t.co/uKIc4txUI9,si,abortolegal https t co ukic4txui9
1,RT @estipozzo: Veo los listados de expositores de mañana por #AbortoLegal. Se viene repitiendo lo siguiente: a favor suelen ser mayoría muj…,si,rt estipozz veo los list de expositor de manan por abortolegal se vien repit lo siguient a favor suel ser mayori muj
2,RT @SofiBotto: HOY! Nuestra coordinadora nacional @raquelvivanco expone en la Cámara de Diputadxs a favor de la ley de #AbortoLegal. \r\nA la…,si,rt sofibott hoy nuestr coordin nacional raquelvivanc expon en la cam de diputadxs a favor de la ley de abortolegal a la
3,RT @SofiBotto: HOY! Nuestra coordinadora nacional @raquelvivanco expone en la Cámara de Diputadxs a favor de la ley de #AbortoLegal. \r\nA la…,si,rt sofibott hoy nuestr coordin nacional raquelvivanc expon en la cam de diputadxs a favor de la ley de abortolegal a la
4,RT @raquelvivanco: Mañana me toca. #AbortoLegal #AhoraEsCuando 💪🏽💚 https://t.co/TackkRzVb4,si,rt raquelvivanc manan me toc abortolegal ahoraescu https t co tackkrzvb4
5,"RT @Red__Accion: #CABA #HOY Tuitazo y actividades en el Congreso por el Aborto Legal, Seguro y Gratuito https://t.co/RnF00ZuDuJ \r\n#AbortoLeg…",si,rt red accion cab hoy tuitaz y activ en el congres por el abort legal segur y gratuit https t co rnf00zuduj abortoleg
6,"RT @ingridurrut: #buenlunes #ATR x el #AbortoLegal Basta de juzgarnos, basta de perseguirnos hoy más que nunca redoblemos la presencia en l…",si,rt ingridurrut buenlun atr x el abortolegal bast de juzg bast de persegu hoy mas que nunc redobl la presenci en l
7,"RT @FranjaMendoza: El Rector @ingdanielpizzi\r\nadhirió a la declaración emitida por el CIN a favor del #Abortolegal, seguro y gratuito.\r\n\r\nhttp…",si,rt franjamendoz el rector ingdanielpizzi adhiri a la declaracion emit por el cin a favor del abortolegal segur y gratuit http
8,"#Argentina #AbortoLegal ""Mi hija no podrá debatir"" https://t.co/Gy67Pspbbh Por @gargantapodero @Red__Accion https://t.co/salDiP4BRs",si,argentin abortolegal mi hij no podr debat https t co gy67pspbbh por gargantapoder red accion https t co saldip4brs
9,RT @TatianaGoransky: Madrid. Hoy ¡Mucha ilu! Presentación doble + amigos + firma a la distancia por #AbortoLegal en Argentina. Gran día. ht…,si,rt tatianagoransky madr hoy much ilu presentacion dobl amig firm a la distanci por abortolegal en argentin gran dia ht


In [211]:
df_tweets.groupby('posicion')['text'].count()

posicion
nan    4   
no     1847
si     2462
Name: text, dtype: int64

In [212]:
df_tweets = df_tweets.drop(columns=['text'])

In [213]:
df_tweets.sample(5)

,posicion,text2
4031,no,rt patriesopran quien estam a favor de nadi men nos reun par celebr la vid este doming en la plaz del agu de 16 a 1
3029,no,rt toyitosal noalabort la buen praxis medic es la que determin la form de actu del personal de salud en condicion complej b
116,si,rt franjamendoz el rector ingdanielpizzi adhiri a la declaracion emit por el cin a favor del abortolegal segur y gratuit http
1565,si,rt carlacarriz no permit que nos rob la palabr vid gran discurs de claudiapineir mas de 200 escritor a favor de ab
2783,no,rt jorgellanosv por much que verborr sobr los ddhh este es un ejempl mas de lo dictador que llev dentr objeciondeconcient


In [214]:
X = df_tweets['text2']
y = df_tweets['posicion']

In [215]:
print(np.max(list(map(lambda x: len(x), X))))
print(np.mean(list(map(lambda x: len(x), X))))
print(np.min(list(map(lambda x: len(x), X))))

130
106.01831671690239
9


In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

In [217]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(strip_accents= 'ascii', stop_words= stopwords_list)
count_vect.fit(X_train)
X_train_counts = count_vect.transform(X_train)

X_train_counts.shape

(2889, 3657)

In [218]:
X_test_counts = count_vect.transform(X_test)

In [219]:
def get_report(test_features,test_target,model, model_name):
    y_pred = model.predict_proba(test_features)
    y_pred_abs = model.predict(test_features)
    Ytest = list(map(lambda x : 1 if(x == "Política") else 0,test_target))
    fpr_log,tpr_log,thr_log = roc_curve(Ytest, y_pred[:,1])
    df_aux = pd.DataFrame(dict(fpr=fpr_log, tpr=tpr_log, thr = thr_log))
    plt.axis([0, 1.01, 0, 1.01])
    plt.xlabel('1 - Specificty')
    plt.ylabel('TPR / Sensitivity')
    plt.title('Curve')
    plt.plot(df_aux['fpr'],df_aux['tpr'])
    plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
    plt.show() 
    print("--------------REPORT MODEL {}--------------\n".format(model_name))
    print("\n Classification Report: \n")
    print(confusion_matrix(test_target, y_pred_abs))
    print("-------------------------------------------\n")

Modelo KNN

In [220]:
model_multinb = MultinomialNB().fit(X_train_counts, y_train)
y_pred_multinb = model_multinb.predict(X_test_counts)

print('Accuracy Multi NB: ',accuracy_score(y_pred_multinb, y_test))

Accuracy Multi NB:  0.9634831460674157


In [221]:
df_results = pd.DataFrame(X_test)
df_results['Prediction'] = y_pred_multinb
df_results.head(5)

,text2,Prediction
2299,abortolegal https t co horojbqyt4,si
3011,rt danyd8 esta zurd con osde mononeuronal te explic porqu abort se dan cuent porqu yo elij dec sialav y noalabort ht,no
2590,rt vickyvillarruel https t co mfaeho97oi noalabort,no
20,muchach y si en el plenari se ocup de los curaspedofil y cierr su cruc sobr el abortolegal https t co kdgnmavqg3,si
3971,rt aciprens con el rez del rosari los fiel sacerdot y religi pedir por el respet a la vid en tod sus etap en venezu,no
